<a href="https://colab.research.google.com/github/shaja-asm/CultiVision_Public/blob/main/CultiVision_Project_Ongoing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import picamera
import subprocess

# Set the number of cameras connected
num_cameras = 4

# Set the output directory for the captured images
output_directory = "/path/to/output/directory/"

# Capture images from all cameras simultaneously
with picamera.PiCamera() as camera:
    # Initialize the camera settings
    camera.resolution = (640, 480)  # Adjust the resolution as needed

    # Capture images from each camera
    for camera_num in range(num_cameras):
        # Set the camera number for Arducam adapter
        subprocess.call(["sudo", "arducamstill", "-t", "1", "-n", str(camera_num)])

        # Wait for the camera to initialize
        time.sleep(2)

        # Capture an image from the current camera
        output_file = f"camera_{camera_num}.jpg"
        camera.capture(output_directory + output_file)
        print(f"Image captured from camera {camera_num}")


In [ ]:
import time
import os
from azure.storage.blob import BlobServiceClient
import cv2
import numpy as np

# Set the connection string and container name for Azure Storage Blob
connection_string = "DefaultEndpointsProtocol=https;AccountName=cultivision;AccountKey=XXXXXXXXXXX;EndpointSuffix=core.windows.net"
container_name = "cvblob"

# Set the output directory for the captured images
output_directory = "store1/"

# Capture images from all cameras simultaneously
for camera_num in range(4):
    # Set the camera number for Arducam adapter
    command = f"sudo arducamstill -t 1 -n {camera_num}"
    os.system(command)
    time.sleep(2)  # Wait for the camera to initialize

    # Capture an image from the current camera
    output_file = f"camera_{camera_num}.jpg"
    output_path = os.path.join(output_directory, output_file)
    capture_command = f"raspistill -o {output_path}"
    os.system(capture_command)

    # Load the captured image
    image = cv2.imread(output_path)

    # Split the image into channels (BGR order for OpenCV)
    blue, green, red, nir = cv2.split(image)

    # Calculate NDVI
    ndvi = (nir.astype(float) - red) / (nir.astype(float) + red)

    # Calculate GNDVI
    gndvi = (nir.astype(float) - green) / (nir.astype(float) + green)

    # Calculate GCI
    gci = green.astype(float) / red

    # Normalize the index values to 0-255 range
    ndvi_norm = cv2.normalize(ndvi, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    gndvi_norm = cv2.normalize(gndvi, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    gci_norm = cv2.normalize(gci, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

    # Save the index images
    output_ndvi = f"ndvi_{camera_num}.jpg"
    output_gndvi = f"gndvi_{camera_num}.jpg"
    output_gci = f"gci_{camera_num}.jpg"

    cv2.imwrite(os.path.join(output_directory, output_ndvi), ndvi_norm)
    cv2.imwrite(os.path.join(output_directory, output_gndvi), gndvi_norm)
    cv2.imwrite(os.path.join(output_directory, output_gci), gci_norm)

    # Upload the captured images and index images to Azure Blob Storage
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=output_file)
    with open(output_path, "rb") as image_file:
        blob_client.upload_blob(image_file)

    blob_ndvi_client = blob_service_client.get_blob_client(container=container_name, blob=output_ndvi)
    with open(os.path.join(output_directory, output_ndvi), "rb") as ndvi_file:
        blob_ndvi_client.upload_blob(ndvi_file)

    blob_gndvi_client = blob_service_client.get_blob_client(container=container_name, blob=output_gndvi)
    with open(os.path.join(output_directory, output_gndvi), "rb") as gndvi_file:
        blob_gndvi_client.upload_blob(gndvi_file)

    blob_gci_client = blob_service_client.get_blob_client(container=container_name, blob=output_gci)
    with open(os.path.join(output_directory, output_gci), "rb") as gci_file:
        blob_gci_client.upload_blob(gci_file)

    print(f"Image captured from camera {camera_num} and indices uploaded to Azure Blob Storage")

# Cleanup: Delete the captured images and index images from the local directory
for camera_num in range(4):
    output_file = f"camera_{camera_num}.jpg"
    output_path = os.path.join(output_directory, output_file)
    os.remove(output_path)

    output_ndvi = f"ndvi_{camera_num}.jpg"
    output_gndvi = f"gndvi_{camera_num}.jpg"
    output_gci = f"gci_{camera_num}.jpg"

    os.remove(os.path.join(output_directory, output_ndvi))
    os.remove(os.path.join(output_directory, output_gndvi))
    os.remove(os.path.join(output_directory, output_gci))
